In [1]:
import os
from selenium import webdriver #Webdriver de Selenium qui permet de contrôler un navigateur
from selenium.webdriver.common.by import By #Permet d'accéder aux différents élements de la page web
from selenium.webdriver.common.keys import Keys # Importe les clefs pour les touches du clavier
from webdriver_manager.chrome import ChromeDriverManager #Assure la gestion du webdriver de Chrome
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import math
import time
from time import sleep 

from tqdm import tqdm # librairie permettant d'indiquer le pourcentage de la boucle for


import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np
gaming = pd.read_csv('vgsales.csv', index_col=0)
gaming.head(5)


,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
Rank,,,,,,,,,,
1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [3]:
gaming_parts = np.array_split(gaming, 16)
gaming_parts[9].head()

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
Rank,,,,,,,,,,
9341,Castlevania Chronicles,PS,2001.0,Platform,Konami Digital Entertainment,0.07,0.05,0.00,0.01,0.13
9342,Suzumiya Haruhi no Tomadoi,PS2,2008.0,Adventure,Banpresto,0.00,0.00,0.13,0.00,0.13
9343,MLB Power Pros 2008,Wii,2008.0,Sports,Konami Digital Entertainment,0.12,0.00,0.01,0.01,0.13
9344,NHL 2K8,X360,2007.0,Sports,Take-Two Interactive,0.11,0.01,0.00,0.01,0.13
9345,Dokapon Kingdom,Wii,2008.0,Role-Playing,Sting,0.12,0.00,0.00,0.01,0.13


In [4]:
def scroll(value): #Définition d'une fonction pour scroller automatiquement
    for i in range(20): #nombre de micro-scroll à effectuer
        driver.execute_script("window.scrollBy(0, {})".format(value))
        time.sleep(0.1) #Temps entre chaque scroll

In [5]:
#scrapping part 10
gaming_JVC=gaming_parts[9]

# Ouvrir une page Google Chrome
driver = webdriver.Chrome(ChromeDriverManager().install()) 
# créer une attente implicite
driver.implicitly_wait(10)

#Lancer une recherche en utilisant un fichier excel ou csv (en ittérant sur une colonne)

for i, row in tqdm(gaming_JVC.iterrows(), total = len(gaming_JVC)):
    
    # Ouvrir l'adresse du site web
    driver.get('https://www.jeuxvideo.com/tous-les-jeux/')
    try:
        # accepter les cookies
        driver.find_element(By.XPATH, "//*[@id='cmp-main']/button[2]").click()
    except:
        pass
    
    #Ajout d'un temps de deux secondes avant de lancer l'action suivante
    time.sleep(1) 
    
    loupe_recherche=driver.find_element(By.CLASS_NAME, 'searchContainer__1f9uG_')
    loupe_recherche.click()

    try:
        barre_recherche_site=driver.find_element(By.CLASS_NAME,"searchInput__38b75-")
        barre_recherche_site.send_keys(row['Name'])
        barre_recherche_site.send_keys(Keys.ENTER)

    except:
        pass

    try:    
        premier_jeu=driver.find_elements(By.CLASS_NAME,"col")[0]
        premier_jeu.click()

        # cliquer sur la bonne plateforme
        boutons_plateforme=driver.find_elements(By.CLASS_NAME,"gameHeaderBanner__platform")
        boutons_plateforme
    except:
        pass

    try:
        for bouton in boutons_plateforme:
            if str(bouton.text) == str(row['Platform']):
                bouton.click()
            else:
                pass
    except:
        pass

    try:
        test_note=driver.find_element(By.CLASS_NAME, "gameCharacteristicsMain__gauge")
        gaming_JVC.loc[i, 'test_note_JVC']=test_note.text
    except:
        gaming_JVC.loc[i, 'test_note_JVC']='no_data'
    
    try:
        avis_count=driver.find_element(By.CLASS_NAME, "gameCharacteristicsMain__count")
        gaming_JVC.loc[i, 'avis_count_JVC']=avis_count.text
    except:
        gaming_JVC.loc[i, 'avis_count_JVC']='no_data'

    try:
        avis_note=driver.find_elements(By.CLASS_NAME, "gameCharacteristicsMain__gauge")[1]
        gaming_JVC.loc[i, 'avis_note_JVC']=avis_note.text
    except:
        gaming_JVC.loc[i, 'avis_note_JVC']='no_data'
    
    scroll(100)

    try:
        game_caracteristique=driver.find_element(By.CLASS_NAME, "gameCharacteristicsDetailed__table")
        gaming_JVC.loc[i, 'description_JVC']=game_caracteristique.text
    except:
        gaming_JVC.loc[i, 'description_JVC']='no_data'
        


[WDM] - Downloading: 100%|██████████| 6.79M/6.79M [00:01<00:00, 4.70MB/s]
100%|██████████| 1037/1037 [8:19:10<00:00, 28.88s/it]  


In [6]:
driver.close()

In [7]:
gaming_JVC.head(10)

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,test_note_JVC,avis_count_JVC,avis_note_JVC,description_JVC
Rank,,,,,,,,,,,,,,
9341,Castlevania Chronicles,PS,2001.0,Platform,Konami Digital Entertainment,0.07,0.05,0.00,0.01,0.13,10,(15),13.3,Editeur(s) / Développeur(s) Konami\nSortie Fra...
9342,Suzumiya Haruhi no Tomadoi,PS2,2008.0,Adventure,Banpresto,0.00,0.00,0.13,0.00,0.13,no_data,no_data,no_data,no_data
9343,MLB Power Pros 2008,Wii,2008.0,Sports,Konami Digital Entertainment,0.12,0.00,0.01,0.01,0.13,--,(1),15,Editeur(s) / Développeur(s) Konami 2K Sports\n...
9344,NHL 2K8,X360,2007.0,Sports,Take-Two Interactive,0.11,0.01,0.00,0.01,0.13,16,(4),18,Editeur(s) / Développeur(s) Kush Games 2K Spor...
9345,Dokapon Kingdom,Wii,2008.0,Role-Playing,Sting,0.12,0.00,0.00,0.01,0.13,12,(9),15,Editeur(s) / Développeur(s) Sting Atlus\nSorti...
9346,Bratz,GBA,2002.0,Platform,Ubisoft,0.10,0.04,0.00,0.00,0.13,--,no_data,no_data,Editeur(s) / Développeur(s) Ubisoft DC Studios...
9347,Go Play: Circus Star,Wii,2009.0,Action,Majesco Entertainment,0.12,0.00,0.00,0.01,0.13,--,no_data,no_data,Editeur(s) / Développeur(s) Majesco N-Fusion I...
9348,King Arthur,PS2,2004.0,Action,Konami Digital Entertainment,0.07,0.05,0.00,0.02,0.13,10,(10),14.7,Editeur(s) / Développeur(s) Krome Studios Kona...
9349,Winter Stars,Wii,2011.0,Sports,Deep Silver,0.06,0.07,0.00,0.01,0.13,10,(2),9.5,Editeur(s) / Développeur(s) 49 Games Deep Silv...


In [ ]:
gaming_JVC.to_csv('gaming_JVC_10.csv')